## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("zayuvalza@gmail.com")  
    to_email = To("zayuvalza@gmail.com")  
    # Content(type of the content, The email message)
    content = Content("text/plain", "This is an important test email")
    # Mail(from, to, subject of the email, content)
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [7]:
'''
run_streamed - allows for real-time streaming of text, bit by bit (like in chat).
Use async for to go through the entire stream of text and print it sequentially (instead of receiving it all at once).
'''
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in simplifying the process of achieving and maintaining SOC 2 compliance.

As you may know, SOC 2 compliance is crucial for building trust with your clients and safeguarding sensitive data. However, the preparation for audits can often be a complex and time-consuming endeavor. That’s where ComplAI comes in. 

Our AI-powered SaaS tool streamlines compliance processes, helping organizations like yours:

- **Automate Documentation**: Reduce manual effort and minimize human error in documentation.
- **Continuous Monitoring**: Stay proactive with real-time insights and alerts on compliance status.
- **Seamless Collaboration**: Facilitate teamwork across various departments for efficient audit preparation.

Would you be open to a brief call this week to discuss how ComplAI can support your compliance

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance Efforts

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name] from ComplAI, where we specialize in helping organizations like yours simplify and expedite SOC 2 compliance processes.

As you know, maintaining compliance can be daunting and time-consuming, often diverting valuable resources away from core business functions. Our AI-powered SaaS tool is designed to automate and streamline compliance workflows, ensuring that you can focus on what matters most—growing your business.

Here are a few ways ComplAI can deliver value to your organization:

1. **Automated Readiness Assessments**: Quickly identify gaps in compliance and receive actionable insights.
2. **Efficient Document Management**: Centralize and manage all necessary documentation with ease.
3. **Simplified Audit Preparation**: Prepare for audits effortlessly, with real-time reporting capabilities.

I would love the opportunity to discuss how our solution can hel

In [9]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [10]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Ready to Make Your SOC2 Compliance a Walk in the Park?

Hey [First Name],

I hope you're ready for a little stroll through the world of SOC2 compliance! 🌳 (Don’t worry—no walking shoes required.)

At ComplAI, we know that audits can feel like navigating a labyrinth: confusing, stressful, and sometimes you come out wondering where you left your sanity. But what if I told you there’s a way to turn that hard maze into a smooth, breezy pathway?

Our AI-powered tool is like having a compliance ninja on your team—stealthy, efficient, and ready to kick audit stress to the curb. We’ll take care of the heavy lifting, leaving you free to focus on more important things, like perfecting your coffee brew or strategizing your next Netflix binge. 🍿

Are you ready to transform your compliance journey? Let’s chat! I promise I won’t fill your inbox with endless jargon—just some good old-fashioned relief from the chaos.

Looking forward to hearing from you!

Best,

[Your Name] 

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [11]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [12]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

<span style="color: orange; font-weight: bold; font-size: 30px;">🤖 Agent() – Fields Overview</span><br>
An `Agent` represents an AI agent (LLM + configuration + tools + guardrails).<br>

<span style="color: pink; font-weight: bold;">Core Fields:</span>
- **`name`** – Name of the agent.
- **`instructions`** – General behavior guidelines (system prompt).
- **`prompt`** – Custom starting prompt (if provided).
- **`handoff_description`** – When/why the agent should hand over to another agent.
- **`handoffs`** – List of defined handoff rules.
- **`model`** – The model to use (e.g., `"gpt-4o-mini"`).

<span style="color: pink; font-weight: bold;">Model Settings (`model_settings`):</span>
- **`temperature`** – Creativity vs. determinism (higher = more creative).
- **`top_p`** – Nucleus sampling (controls probability mass).
- **`frequency_penalty`** – Penalize word repetition.
- **`presence_penalty`** – Encourage introducing new ideas.
- **`tool_choice`** – Control automatic vs. forced tool selection.
- **`parallel_tool_calls`** – Allow multiple tools in parallel.
- **`truncation`** – How to shorten overly long responses.
- **`max_tokens`** – Limit for response length.
- **`reasoning`** – Control reasoning depth (if supported).
- **`metadata`** – Extra metadata fields.
- **`store`** – Whether to persist state across conversations.
- **`include_usage`** – Include usage statistics (token counts).
- **`extra_query / extra_body / extra_headers / extra_args`** – Custom API overrides.

<span style="color: pink; font-weight: bold;">Tools and Integrations:</span>
- **`tools`** – List of tools (FunctionTools or MCP tools).
- **`mcp_servers`** – Connections to MCP (Model Context Protocol) servers.
- **`mcp_config`** – Extra configuration for MCP.

<span style="color: pink; font-weight: bold;">Guardrails & Output:</span>
- **`input_guardrails`** – Filters/validators for user input.
- **`output_guardrails`** – Filters/validators for model output.
- **`output_type`** – Expected output type (`"json"`, `"text"`, `"markdown"`, etc.).

<span style="color: pink; font-weight: bold;">Advanced:</span>
- **`hooks`** – Functions triggered before/after model calls.
- **`tool_use_behavior`** – Defines behavior after tool use (e.g., `"run_llm_again"`).
- **`reset_tool_choice`** – Whether to reset tool choice after use.

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [13]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("zayuvalza@gmail.com")  # Change to your verified sender
    to_email = To("zayuvalza@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [14]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029297800D60>, strict_json_schema=True, is_enabled=True)

<span style="color: orange; font-weight: bold; font-size: 30px;">🛠️ FunctionTool() – Fields Overview</span><br>
A `FunctionTool` wraps a Python function so the agent can invoke it.<br>

<span style="color: pink; font-weight: bold;">Core Fields:</span>
- **`name`** – Tool name (e.g., `"send_email"`).
- **`description`** – Text description of what the tool does.

<span style="color: pink; font-weight: bold;">Parameters:</span>
- **`params_json_schema`** – JSON Schema defining expected arguments:
  - `properties` – Available parameters with types.
  - `required` – Parameters that must be provided.
  - `title` – Schema title.
  - `type` – Usually `"object"`.
  - `additionalProperties` – Whether extra params are allowed.

<span style="color: pink; font-weight: bold;">Behavior:</span>
- **`on_invoke_tool`** – Actual Python function executed when tool is called.
- **`strict_json_schema`** – If `True`, only schema-defined params are allowed.
- **`is_enabled`** – Whether the tool is active/usable by the agent.

### And you can also convert an Agent into a tool

In [15]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000292972E8EA0>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [16]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002929563DC60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029297931DA0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

## And now it's time for our Sales Manager - our planning agent

In [17]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [18]:
sales_manager

Agent(name='Sales Manager', instructions='\nYou are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.\n\nFollow these steps carefully:\n1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.\n\n2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.\n\n3. Use the send_email tool to send the best email (and only the best email) to the user.\n\nCrucial Rules:\n- You must use the sales agent tools to generate the drafts — do not write them yourself.\n- You must send ONE email using the send_email tool — never more than one.\n', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_token

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



עד עכשיו למדנו שאפשר להפוך Agent ל־Tool באמצעות .as_tool().

### Handoff:

Tool = כלי קטן שבו ה־ Agent משתמש (בקשה–תשובה).

Handoff = ה־ Agent בעצם מוסר שליטה מלאה ל־ Agent אחר.

כלומר הוא לא רק "משתמש" בו, אלא מעביר אליו אחריות על המשימה.

מרגע שה־ handoff מתבצע, הזרימה לא חוזרת ל־ Agent המקורי.

ההבדלים העיקריים:

1) Agent כ־ Tool → שימוש מקומי, כמו פונקציה שעוזרת. השליטה חוזרת ל־ Agent הקורא.

2) Handoff → העברת אחריות מלאה ל־ Agent אחר, השליטה לא חוזרת.

In [19]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [20]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("zayuvalza@gmail.com")  # Change to your verified sender
    to_email = To("zayuvalza@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [21]:
tools = [subject_tool, html_tool, send_html_email]

In [22]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000292972E8E00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029297933060>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to 

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools, # subject_tool, html_tool, send_html_email
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")
    # handoff_description is essentially a "calling card" –
    # if another agent wants to pass on a task, they know that this Agent can be in it.
    # It's similar to the tool description, only at the delivery level.


In [24]:
emailer_agent

Agent(name='Email Manager', instructions='You are an email formatter and sender. You receive the body of an email to be sent. You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. Finally, you use the send_html_email tool to send the email with the subject and HTML body.', prompt=None, handoff_description='Convert an email to HTML and send it', handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subjec

### Now we have 3 tools and 1 handoff

In [25]:
tools = [tool1, tool2, tool3] # 3 sales agents
handoffs = [emailer_agent] # handoff agent that itself has 3 tool3 - subject_tool, html_tool, send_html_email
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002929563DC60>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029297931DA0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools, # 3 sales agents that write the text emails
    handoffs=handoffs, # sales_manager has handoff agent (emailer_agent) that make subject, design to HTML and send the emails.
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [27]:
sales_manager

Agent(name='Sales Manager', instructions="\nYou are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.\n\nFollow these steps carefully:\n1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.\n\n2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.\nYou can use the tools multiple times if you're not satisfied with the results from the first try.\n\n3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.\n\nCrucial Rules:\n- You must use the sales agent tools to generate the drafts — do not write them yourself.\n- You must hand off exactly ONE email to the Email Manager — never more than one.\n", prompt=None, handoff_description=None, handoffs=[Agent(name='Email Manager', instruc

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.